Import the necessary modules for the **PumpController**

In [1]:
from pump_controller import PumpController, get_serial_port, list_serial_ports
from pump_controller import visualize_rgb, visualize_candidates
import time
import numpy as np

### Calibrations

Now we initialize the calibratebot. We need to figure out which port that the controller is connected to your computer. The `get_serial_port` should automatically do this for you, but things have a way of failing when you need them not to. That is why you can call the `list_serial_ports` function to see all the ports on your computer, and you can simply use the correct port as a string input instead of using the `get_serial_port` function. The `cell_volume` and `drain_time` properties are already defined to 15 mL and 15 seconds, respectively, but you have the option of changing them here. Notice that a folder called *logs* is created with a file with the current timestamp in it - this is where the colors that you mix on this controller in this session will be stored.


We start with the `config_template.json` file as the `config_file` to begin with, but we will make our own in this tutorial.

In [4]:
calibratebot = PumpController(ser_port = get_serial_port(), cell_volume = 15, drain_time = 15, config_file = 'config_files/config_template.json')

Serial port /dev/cu.usbserial-210 opened at baud rate 9600
Arduino is ready


Before doing anything else, we should purge all the pumps. This means filling all of the tubes with their respective liquids. We do this by using the `purge_pump` function, which takes the pump name ('R', 'G', 'B', 'Y', 'W', 'D') and time as variables. Do this one-by-one until all tubes are filled with liquid. The drain tubes of course do not need to be purged. If you are filling up the test cell, you can then drain the test cell using the `drain` function which by itself drains the whole cell for the defined drain time in the previous cell, but you can also define a custom drain time here. If needed you can also flush the cell with water using the `flush` function and then drain again.

In [5]:
calibratebot.purge_pump('R', 3)

# calibratebot.purge_pump('G', 5)
# calibratebot.purge_pump('B', 5)
# calibratebot.purge_pump('Y', 5)
# calibratebot.purge_pump('W', 5)

# Drain:
# calibratebot.drain()

# Drain for custom time:
# calibratebot.drain(drain_time = 10)

# Flush:
# calibratebot.flush()

Sent from PC -- TEST STR -- <Mix,2,3>
Reply Received  Msg Mix,2,3 Time 99


Now we want to calibrate the pumps. We need to figure out how many mL of water comes out of the pump for every second that we run it. We do this by running the pump for certains amounts of times and weigh the amount of liquid that is output. We can then do a linear regression in Excel to find the slope and the offset for each pump. As each pump is different, these values will also be different. 


The `calibration_test` function cycles through different amounts of time of running the pumps, and waits for 5 seconds. In these 5 seconds, your job is to read the measurement on the weighing scale and note it down in the excel sheet.

In [118]:
times = [0.1, 0.2, 0.4, 0.8, 1.0, 1.6, 2.0, 2.5, 3.0, 5.0, 7.0, 8.0]

def calibration_test(pump, times):
    for t in times:
        calibratebot.purge_pump(pump, t)
        print(f"sleeping after {t} on {pump}")
        print("-----------")
        print()
        
        time.sleep(t + 5.0)

Let us run the calibration now!

In [121]:
calibration_test('R', times)

# calibration_test('G', times)
# calibration_test('B', times)
# calibration_test('Y', times)
# calibration_test('W', times)
# calibration_test('D', times)

Sent from PC -- TEST STR -- <Mix,6,0.1>
Reply Received  Msg Mix,6,0.1 Time 1763
sleeping after 0.1 on W
-----------

Sent from PC -- TEST STR -- <Mix,6,0.2>
Reply Received  Msg Mix,6,0.2 Time 1773
sleeping after 0.2 on W
-----------

Sent from PC -- TEST STR -- <Mix,6,0.4>
Reply Received  Msg Mix,6,0.4 Time 1783
sleeping after 0.4 on W
-----------

Sent from PC -- TEST STR -- <Mix,6,0.8>
Reply Received  Msg Mix,6,0.8 Time 1794
sleeping after 0.8 on W
-----------

Sent from PC -- TEST STR -- <Mix,6,1.0>
Reply Received  Msg Mix,6,1.0 Time 1805
sleeping after 1.0 on W
-----------

Sent from PC -- TEST STR -- <Mix,6,1.6>
Reply Received  Msg Mix,6,1.6 Time 1817
sleeping after 1.6 on W
-----------

Sent from PC -- TEST STR -- <Mix,6,2.0>
Reply Received  Msg Mix,6,2.0 Time 1830
sleeping after 2.0 on W
-----------

Sent from PC -- TEST STR -- <Mix,6,2.5>
Reply Received  Msg Mix,6,2.5 Time 1844
sleeping after 2.5 on W
-----------

Sent from PC -- TEST STR -- <Mix,6,3.0>
Reply Received  Msg Mix,

Take these calibration slope and intercept values for each of the pumps, and note them down in the `config.json` file